In [ ]:
#Import modules
import act
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import urllib3
import pandas as pd
import warnings
from metpy.units import pandas_dataframe_to_unit_arrays, units
from datetime import datetime
warnings.simplefilter('ignore')

In [ ]:
#What are the possible crop types?
df = pd.read_csv('cdl_2022_stat_clip_20230609045923_1995493459.csv')
data = np.array(df['Category'])
data

In [ ]:
# Set your ARM Live data username and password.
username = 'username'
token = 'token'
n = input(f'Facility Number:')
# Set the datastream and dates for download.  Let's just look at one week to start
# If you don't know the datastream you can always download through data discovery as well
# https://adc.arm.gov/discovery/#/
# You can also easily change the facility (E14) to other ones as well
datastream = f'sgp30qcecorE{int(n)}.s1'
startdate = input(f'Input Start Date (YYYYMMDD):')
enddate = input(f'Input End Date (YYYYMMDD):')

# Download data using the webservice
qcecor_files = act.discovery.download_data(username, token, datastream, startdate, enddate)

# Download ECOR data using the webservice
datastream = f'sgp30ecorE{int(n)}.b1'
ecor_files = act.discovery.download_data(username, token, datastream, startdate, enddate)

In [ ]:
# Reading in data to an xarray dataset is very easy using ACT
# ARM has some standards that can cause issues with the xarray reader at times
ds = act.io.armfiles.read_netcdf(qcecor_files)
ds_ecor = act.io.armfiles.read_netcdf(ecor_files)
ds_ecor

In [ ]:
#Get variables in the proper format
wspd = ds_ecor['wind_spd'].values
wdir = ds_ecor['wind_dir'].values
ustar = ds_ecor['ustar'].values
h = ds['sensible_heat_flux'].values
lv = ds['latent_heat_flux'].values
fc = ds_ecor['fc'].values
lat = ds['lat'].values
lon = ds['lon'].values
times = ds_ecor['time'].values
year = times.astype('datetime64[Y]').astype(int) + 1970

In [ ]:
#Subset the times
time=[]
b = input(f'Decide how to subset your data\n'
         'For weekly data, enter 336\n'
         'For daily data, enter 48\n'
         'For twice-daily data, enter 24\n'
         'For hourly data, enter 2\n'
         'For half-hourly data, enter 1\n'
         'Enter your number here:')
c = input(f'Decide the hour offset:')
for t in range(2*int(c), len(times), int(b)):
    time.append(times[t])
time = np.array(time)
print(time)

In [ ]:
#Subset the data and put into a text file
outfile = open(f'flux_data.txt', 'w')
for a in range(2*int(c), len(times), int(b)):
    outfile.writelines(f'{(wdir[a]):6.2f}  {(wspd[a]):.2f}  {(h[a]):6.2f}  {(lv[a]):6.2f}  {(fc[a]):5.2f}  {(lat[a]):.2f}  {(lon[a]):.2f}')
    outfile.writelines(f'\n')
outfile.close()
ds1 = np.loadtxt('flux_data.txt')
ds1

In [ ]:
#Turn the text file into an xarray dataset
ds2 = xr.Dataset(
    data_vars=dict(
        sensible_heat_flux=(["time"], ds1[0:,2]),
        latent_heat_flux=(["time"], ds1[0:,3]),
        wdir2=(["time"], ds1[0:,0]),
        wspd2=(["time"], ds1[0:,1]),
        co2_flux=(["time"], ds1[0:,4]),
    ),
    coords=dict(
        lat=ds1[0:,5],
        lon=ds1[0:,6],
    ),
    attrs=dict(description="Flux Data"),
)
ds2['time'] = time
ds2

In [ ]:
#Turn the data into individual variables
wdir2 = ds2['wdir2']
wspd2 = ds2['wspd2']
sensible_heat_flux = ds2['sensible_heat_flux']
latent_heat_flux = ds2['latent_heat_flux']
co2_flux = ds2['co2_flux'] 

In [ ]:
# We can use xarray funcationality to quickly plot up the data
ds2['sensible_heat_flux'].plot()

In [ ]:
ds2['latent_heat_flux'].plot()

In [ ]:
# Let's just make sure the wind data looks good as well
ds2['wspd2'].plot()

In [ ]:
# First, let's build a function that has the "Good" fetch directions defined
def get_arm_fetch(site, fac):
    """
    Each SGP ECOR site has specific directions where the fetch is good
    These ranges list are of good fetch directions
    """
    ranges=[]
    if site == 'sgp':
        if fac == 'E1':  #ECOR
            ranges = [[0, 53], [120, 360]]
        if fac == 'E2':  # EBBR
            ranges = [[71, 137], [223, 289]]
        if fac == 'E3':  #ECOR
            ranges = [[0, 48], [132, 260]]
        if fac == 'E4':  # EBBR
            ranges = [[0, 158], [202, 360]]
        if fac == 'E5':  #ECOR
            ranges = [[80, 154], [154, 260]]
        if fac == 'E6':  #ECOR
            ranges = [[0, 360]]
        if fac == 'E7':  # EBBR
            ranges = [[0, 244], [296, 360]]
        if fac == 'E8':  # EBBR
            ranges = [[0, 224], [314, 360]]
        if fac == 'E9':  # EBBR
            ranges = [[0, 360]]
        if fac == 'E10':  #ECOR
            ranges = [[0, 360]]
        if fac == 'E11':  # EBBR
            ranges = [[0, 360]]
        if fac == 'E12':  # EBBR
            ranges = [[0, 360]]
        if fac == 'E13':  # EBBR
            ranges = [[0, 52], [142, 194], [328, 360]]
        if fac == 'E14':  #ECOR
            ranges = [[129, 265], [352, 360], [0, 85]]
        if fac == 'E15':  # EBBR
            ranges = [[133, 360]]
        if fac == 'E16':  #ECOR
            ranges = [[134, 269], [334, 360]]
        if fac == 'E18':  # EBBR
            ranges = [[138, 325]]
        if fac == 'E19':  # EBBR
            ranges = [[0, 133], [151, 360]]
        if fac == 'E20':  # EBBR
            ranges = [[0, 230], [310, 360]]
        if fac == 'E21':  #ECOR
            ranges = [[30, 360]]
        if fac == 'E22':  # EBBR
            ranges = [[0, 49], [139, 360]]
        if fac == 'E24':  #ECOR
            ranges = [[80, 280]]
        if fac == 'E25':  # EBBR
            ranges = [[30, 360]]
        if fac == 'E26':  # EBBR
            ranges = [[0, 33], [243, 360]]
        if fac == 'E27':  # EBBR
            ranges = [[20,156]]
        if fac == 'E31':  #ECOR
            ranges = [[100, 200], [30, 80]]
        if fac == 'E32':  # EBBR
            ranges = [[0,360]]
        if fac == 'E33':  #ECOR
            ranges = [[100, 300], [40, 80]]
        if fac == 'E34':  # EBBR
            ranges = [[0,360]]
        if fac == 'E35':  # EBBR
            ranges = [[0,360]]
        if fac == 'E36':  # EBBR
            ranges = [[0,360]]
        if fac == 'E37':  #ECOR
            ranges = [[135, 260], [280, 310]]
        if fac == 'E38':  #ECOR
            ranges = [[150, 260]]
        if fac == 'E39':  #ECOR
            ranges = [[100, 260], [280, 360], [0, 80]]
        if fac == 'E40':  # EBBR
            ranges = [[0,360]]
        if fac == 'E41':  #ECOR
            ranges = [[100, 260], [280, 360], [0, 80]]

    return ranges

In [ ]:
urllib3.disable_warnings()
# Get good fetch ranges for this site
site = ds.attrs['site_id']
fac = ds.attrs['facility_id']
ranges = get_arm_fetch(site, fac)
#Get the crop types
crop = []

for i, d in enumerate(wdir2):
    lat2, lon2 = act.utils.geo_utils.destination_azimuth_distance(lat[0], lon[0], d, 100.)
    if np.isnan(lat2) or np.isnan(lon2):
        crop.append(np.nan)
    else:
        crop.append(act.discovery.get_cropscape.croptype(lat2, lon2, year[i]))
        print(crop[i])

In [ ]:
#Create an array of the crop types, then add it to our dataset
da = xr.DataArray(data=crop, dims=ds2['wdir2'].dims, name='crop_type')
ds2['crop_type']=da

In [ ]:
#What crop types are we working with?
crop_list = []
for j in range(0, 89):
    if data[j] in da:
        if len(np.where(da == data[j])[0]) > 1:
            crop_list.append(data[j])
crop_list

In [ ]:
#Flux-Time Graphs
for x in range(0, len(crop_list)):     
    y = np.where(da == crop_list[x])
    plt.figure(figsize=(16, 5))
    ax = plt.subplot(211)
    ax.scatter(time[y], sensible_heat_flux[y], color = 'red', label = 'Sensible Heat Flux')
    ax.scatter(time[y], latent_heat_flux[y], color = 'blue', label = 'Latent Heat Flux')
    ax.plot(time[y], sensible_heat_flux[y], color = 'darkred')
    ax.plot(time[y], latent_heat_flux[y], color = 'darkblue')
    ax.set_ylabel('Heat Flux (W/m^2)')
    ax.set_title(f"Fluxes Over Time for {crop_list[x]}")
    ax.legend()
    ax = plt.subplot(212)
    ax.scatter(time[y], co2_flux[y], color = 'green', label = 'CO2 Flux')
    ax.plot(time[y], co2_flux[y], color = 'darkgreen')
    ax.set_xlabel('Time')
    ax.set_ylabel('CO2 Flux (umol/(sm^2))')
    ax.legend()

In [ ]:
#Histograms
for x in range(0, len(crop_list)):  
    ds3 = ds2.where(ds2.crop_type == f'{crop_list[x]}')
    hist = act.plotting.HistogramDisplay({f'{crop_list[x]}': ds3}, subplot_shape=(1,3), figsize=(19, 5))
    hist.plot_stairstep_graph('sensible_heat_flux', dsname=f'{crop_list[x]}', subplot_index=(0,0), label=f'{crop_list[x]}')
    hist.plot_stairstep_graph('latent_heat_flux', dsname=f'{crop_list[x]}', subplot_index=(0,1), label=f'{crop_list[x]}')
    hist.plot_stairstep_graph('co2_flux', dsname=f'{crop_list[x]}', subplot_index=(0,2), label=f'{crop_list[x]}')

In [ ]:
#Polar Coordinate Graphs
for x in range(0, len(crop_list)):

    ds3 = ds2.where(ds2.crop_type == f'{crop_list[x]}')
    
    display = act.plotting.WindRoseDisplay({f'{crop_list[x]}': ds3},
                                           subplot_shape=(1,3), figsize=(16,6))
    display.plot_data('wdir2', 'wspd2', 'sensible_heat_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,0), contour_type='mean', num_data_bins=10,
    clevels=21)
    display.plot_data('wdir2', 'wspd2', 'latent_heat_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,1), contour_type='mean', num_data_bins=10,
    clevels=21)
    display.plot_data('wdir2', 'wspd2', 'co2_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,2), contour_type='mean', num_data_bins=10,
    clevels=21)

In [ ]:
#Polar Coordinate Graphs
for x in range(0, len(crop_list)):

    ds3 = ds2.where(ds2.crop_type == f'{crop_list[x]}')
    
    display = act.plotting.WindRoseDisplay({f'{crop_list[x]}': ds3},
                                           subplot_shape=(1,3), figsize=(16,6))
    display.plot_data('wdir2', 'wspd2', 'sensible_heat_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,0), contour_type='mean')
    display.plot_data('wdir2', 'wspd2', 'latent_heat_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,1), contour_type='mean')
    display.plot_data('wdir2', 'wspd2', 'co2_flux', num_dirs=12, plot_type='contour', 
                      dsname=f'{crop_list[x]}', subplot_index=(0,2), contour_type='mean')